# Projeto 1 - Ciência dos Dados

Nome: Arthur Gomes Chieppe

Nome: João Pedro Di Jardim Marquez

**Atenção:** Serão permitidos grupos de três pessoas, mas com uma rubrica mais exigente. Grupos deste tamanho precisarão fazer um questionário de avaliação de trabalho em equipe

___
Carregando algumas bibliotecas:

In [1]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os
from math import *
from emoji import UNICODE_EMOJI

**Em `filename`, coloque o nome do seu arquivo de dados!**

In [2]:
import os

filename = 'PS5.xlsx'
if filename in os.listdir():
    print(f'Encontrei o arquivo {filename}, tudo certo para prosseguir com a prova!')
else:
    print(f'Não encontrei o arquivo {filename} aqui no diretório {os.getcwd()}, será que você não baixou o arquivo?')

Encontrei o arquivo PS5.xlsx, tudo certo para prosseguir com a prova!


Carregando a base de dados com os tweets classificados como relevantes e não relevantes:

In [3]:
train = pd.read_excel(filename, sheet_name = 'Treinamento')
train.head(5)

,Tweets,Relevância:
0,@immandy4 o arquiteto do ps5 disse que ele ser...,1
1,o vídeo que eu fiz pro evento tá no youtube tb...,0
2,@chief117_com_br @vingadorbrambz enquanto o da...,0
3,possivelmente hoje sai o valor e a data de lan...,1
4,"@lealemy_ sim, mas os produtos da sony vão par...",1


In [4]:
test = pd.read_excel(filename, sheet_name = 'Teste')
test.head(5)


,Tweets,Relevância:
0,"sony nega, mas ações continuam em queda após r...",1
1,hoje tem \n\n#ps5,0
2,@prodigyrr não faça isso. ps5 ta uma completa ...,0
3,"hoje lançam o preço do ps5, tô só por saber qu...",1
4,"@1playergirl bom, nesse momento me sinto num h...",0


___
## Classificador automático de sentimento


Faça aqui uma descrição do seu produto e o que considerou como relevante ou não relevante na classificação dos tweets.

Nosso produto trata-se do console de videogames PlayStation5. Quando extraímos os Tweets, havia uma espera coletiva acerca da livestream de lançamento do novo PlayStation 5. Assim , consideramos relevante especulações acerca do preço do novo console, bem como opiniões dos usuários sobre seu design. Ademais, consideramos algumas comparações entre o novo PlayStation e Xbox como relevantes, porém somente aquelas bem fundamentadas e intencionadas, já que consideramos irrelevantes comparações vagas e com discurso de ódio. Anúncios feitos acerca da live também foram consideradas como relevantes, com exceção de anúncios que tinham como objetivo somente a autopromoção dos usuários.

___
### Montando um Classificador Naive-Bayes

Considerando apenas as mensagens da planilha Treinamento, ensine  seu classificador.

In [5]:
#Limpando caracteres:
def limpa_tweets(dataframe):
    train_limpo = dataframe.iloc[:, 0]
    ls= []
    for w in train_limpo:
        w = w.casefold()
        w = w.replace(":",'')
        w = w.replace('"','')
        w = w.replace("'",'')
        w = w.replace("(",'')
        w = w.replace(")",'')
        w = w.replace('!','')
        w = w.replace('...','')
        w = w.replace(',','')
        w = w.replace('\n','')
        w = w.replace('/','')
        w = w.replace(' a ',' ')
        w = w.replace(' e ',' ')
        w = w.replace(' o ',' ')
        w = w.replace(' de ',' ')
        w = w.replace(' do ',' ')
        w = w.replace(' no ',' ')
        w = w.replace(' na ',' ')
        w = w.replace(' os ',' ')
        w = w.replace(' que ',' ')
        w = w.replace(' q ',' ')
        w = w.replace(' as ',' ')
        w = w.replace(' ps5 ',' ')
        w = w.replace(' - ',' ')
        w = ''.join(' ' + char+' ' if char in UNICODE_EMOJI else char for char in w)
        w = w.replace('  ',' ')
        w = w.strip()
        ls.append(w)
    dataframe['Tweets'] = ls
    return dataframe
train = limpa_tweets(train)


In [6]:
#train.to_excel(r'PS5Emoji.xlsx', sheet_name='Treinamento', index = False)

## Classificador:
Precisamos de:
P(R|T) = P(T|R) * P(R)/ P(T)




In [7]:
#Calculando Probabilidades:
total_portugues = 50000 #Consideramos todas as palavras do portugues comumente usadas no twitter para maximizar smoothing.
def treinamento(dataframe):
    Rele = pd.Series(dtype=object)
    Irrele = pd.Series(dtype=object)
    i = 0
    for s in dataframe['Tweets']:
        lsWords = pd.Series(s.split())
        if dataframe['Relevância:'][i] == 1:
            Rele = Rele.append(lsWords)
        else:
            Irrele = Irrele.append(lsWords)
        i += 1
    
    #Calcaulando probabilidade em tweets relevantes:
    lsRele = []
    for v in Rele.value_counts():
        lsRele.append(abs(log10((v+1)/(total_portugues+Rele.count()))))
    lsKeysRele = Rele.value_counts().index.tolist()
    dic_rele = {lsKeysRele[i]: lsRele[i] for i in range(len(lsKeysRele))} 
    
    lsIrrele = []
    for v in Irrele.value_counts():
        lsIrrele.append(abs(log10((v+1)/(total_portugues+Irrele.count()))))
    lsKeysIrrele = Irrele.value_counts().index.tolist()
    dic_irrele = {lsKeysIrrele[i]: lsIrrele[i] for i in range(len(lsKeysIrrele))}
    print(dic_irrele)
    return [dic_rele, dic_irrele, Rele, Irrele]



In [8]:
treino = treinamento(train)
dic_rele = treinamento(train)[0]
dic_irrele = treinamento(train)[1]
Rele = treinamento(train)[2]
Irrele = treinamento(train)[3]

def classifica(dataframe, dic_rele, dic_irrele, Rele, Irrele):
    lsClassificacao = []
    for t in dataframe['Tweets']:
        lsWord = t.split()
        lsRele = []
        lsIrrele = []
        for w in lsWord:
            if w in dic_rele:
                lsRele.append(dic_rele[w])
            if w in dic_irrele:
                lsIrrele.append(dic_irrele[w])
            else:
                lsRele.append(1/(total_portugues+Rele.count()))
                lsIrrele.append(1/(total_portugues+Irrele.count()))
        if np.sum(lsRele) >= np.sum(lsIrrele):
            lsClassificacao.append(1)
        else:
            lsClassificacao.append(0)
    return lsClassificacao
teste_treinamento = pd.Series(classifica(train, dic_rele, dic_irrele, Rele, Irrele))
teste_treinamento.value_counts()
train['Classificação'] = teste_treinamento
pd.set_option("display.max_rows", None, "display.max_columns", None)


{'um': 2.9733587998863977, 'é': 2.9811841373983543, 'dia': 3.0494489689568804, 'bom': 3.153345102825603, 'pra': 3.190067909850343, 'não': 3.216396848572692, '#ps5': 3.244425572172936, 'eu': 3.244425572172936, 'hoje': 3.244425572172936, 'vai': 3.259148828993642, 'da': 3.341335585180992, 'com': 3.3598189908750054, 'me': 3.3598189908750054, 'evento': 3.3598189908750054, 'uma': 3.399327532158679, 'tem': 3.399327532158679, 'para': 3.442793225939769, 'mais': 3.466274321789292, 'se': 3.4910979055143243, 'xbox': 3.545455567836917, 'meu': 3.6076034745857615, 'sony': 3.6076034745857615, 'isso': 3.6076034745857615, 'ps5': 3.6423655808449733, 'ter': 3.680154141734373, 'preço': 3.680154141734373, 'tá': 3.680154141734373, 'já': 3.680154141734373, 'por': 3.680154141734373, 'ela': 3.680154141734373, 'mas': 3.680154141734373, 'minha': 3.721546826892598, 'em': 3.721546826892598, 'dar': 3.721546826892598, '🖕': 3.767304317453273, 'acordei': 3.767304317453273, 'porra': 3.767304317453273, 'veio': 3.76730431

{'um': 2.9733587998863977, 'é': 2.9811841373983543, 'dia': 3.0494489689568804, 'bom': 3.153345102825603, 'pra': 3.190067909850343, 'não': 3.216396848572692, '#ps5': 3.244425572172936, 'eu': 3.244425572172936, 'hoje': 3.244425572172936, 'vai': 3.259148828993642, 'da': 3.341335585180992, 'com': 3.3598189908750054, 'me': 3.3598189908750054, 'evento': 3.3598189908750054, 'uma': 3.399327532158679, 'tem': 3.399327532158679, 'para': 3.442793225939769, 'mais': 3.466274321789292, 'se': 3.4910979055143243, 'xbox': 3.545455567836917, 'meu': 3.6076034745857615, 'sony': 3.6076034745857615, 'isso': 3.6076034745857615, 'ps5': 3.6423655808449733, 'ter': 3.680154141734373, 'preço': 3.680154141734373, 'tá': 3.680154141734373, 'já': 3.680154141734373, 'por': 3.680154141734373, 'ela': 3.680154141734373, 'mas': 3.680154141734373, 'minha': 3.721546826892598, 'em': 3.721546826892598, 'dar': 3.721546826892598, '🖕': 3.767304317453273, 'acordei': 3.767304317453273, 'porra': 3.767304317453273, 'veio': 3.76730431

{'um': 2.9733587998863977, 'é': 2.9811841373983543, 'dia': 3.0494489689568804, 'bom': 3.153345102825603, 'pra': 3.190067909850343, 'não': 3.216396848572692, '#ps5': 3.244425572172936, 'eu': 3.244425572172936, 'hoje': 3.244425572172936, 'vai': 3.259148828993642, 'da': 3.341335585180992, 'com': 3.3598189908750054, 'me': 3.3598189908750054, 'evento': 3.3598189908750054, 'uma': 3.399327532158679, 'tem': 3.399327532158679, 'para': 3.442793225939769, 'mais': 3.466274321789292, 'se': 3.4910979055143243, 'xbox': 3.545455567836917, 'meu': 3.6076034745857615, 'sony': 3.6076034745857615, 'isso': 3.6076034745857615, 'ps5': 3.6423655808449733, 'ter': 3.680154141734373, 'preço': 3.680154141734373, 'tá': 3.680154141734373, 'já': 3.680154141734373, 'por': 3.680154141734373, 'ela': 3.680154141734373, 'mas': 3.680154141734373, 'minha': 3.721546826892598, 'em': 3.721546826892598, 'dar': 3.721546826892598, '🖕': 3.767304317453273, 'acordei': 3.767304317453273, 'porra': 3.767304317453273, 'veio': 3.76730431

{'um': 2.9733587998863977, 'é': 2.9811841373983543, 'dia': 3.0494489689568804, 'bom': 3.153345102825603, 'pra': 3.190067909850343, 'não': 3.216396848572692, '#ps5': 3.244425572172936, 'eu': 3.244425572172936, 'hoje': 3.244425572172936, 'vai': 3.259148828993642, 'da': 3.341335585180992, 'com': 3.3598189908750054, 'me': 3.3598189908750054, 'evento': 3.3598189908750054, 'uma': 3.399327532158679, 'tem': 3.399327532158679, 'para': 3.442793225939769, 'mais': 3.466274321789292, 'se': 3.4910979055143243, 'xbox': 3.545455567836917, 'meu': 3.6076034745857615, 'sony': 3.6076034745857615, 'isso': 3.6076034745857615, 'ps5': 3.6423655808449733, 'ter': 3.680154141734373, 'preço': 3.680154141734373, 'tá': 3.680154141734373, 'já': 3.680154141734373, 'por': 3.680154141734373, 'ela': 3.680154141734373, 'mas': 3.680154141734373, 'minha': 3.721546826892598, 'em': 3.721546826892598, 'dar': 3.721546826892598, '🖕': 3.767304317453273, 'acordei': 3.767304317453273, 'porra': 3.767304317453273, 'veio': 3.76730431

{'um': 2.9733587998863977, 'é': 2.9811841373983543, 'dia': 3.0494489689568804, 'bom': 3.153345102825603, 'pra': 3.190067909850343, 'não': 3.216396848572692, '#ps5': 3.244425572172936, 'eu': 3.244425572172936, 'hoje': 3.244425572172936, 'vai': 3.259148828993642, 'da': 3.341335585180992, 'com': 3.3598189908750054, 'me': 3.3598189908750054, 'evento': 3.3598189908750054, 'uma': 3.399327532158679, 'tem': 3.399327532158679, 'para': 3.442793225939769, 'mais': 3.466274321789292, 'se': 3.4910979055143243, 'xbox': 3.545455567836917, 'meu': 3.6076034745857615, 'sony': 3.6076034745857615, 'isso': 3.6076034745857615, 'ps5': 3.6423655808449733, 'ter': 3.680154141734373, 'preço': 3.680154141734373, 'tá': 3.680154141734373, 'já': 3.680154141734373, 'por': 3.680154141734373, 'ela': 3.680154141734373, 'mas': 3.680154141734373, 'minha': 3.721546826892598, 'em': 3.721546826892598, 'dar': 3.721546826892598, '🖕': 3.767304317453273, 'acordei': 3.767304317453273, 'porra': 3.767304317453273, 'veio': 3.76730431

In [9]:
teste_treinamento.value_counts()

1    150
0    150
dtype: int64

In [10]:
#Comparando com as entradas manualmente digitadas:
train['Relevância:'].value_counts()

0    151
1    149
Name: Relevância:, dtype: int64

In [11]:
pd.crosstab(train['Relevância:'], train['Classificação'], normalize='index')

Classificação,0,1
Relevância:,,
0,0.986755,0.013245
1,0.006711,0.993289


___
### Verificando a performance do Classificador

Agora você deve testar o seu classificador com a base de Testes.

In [12]:
teste_teste = pd.Series(classifica(test, dic_rele, dic_irrele, Rele, Irrele))
test['Classificação'] = teste_teste
test

,Tweets,Relevância:,Classificação
0,"sony nega, mas ações continuam em queda após r...",1,1
1,hoje tem \n\n#ps5,0,1
2,@prodigyrr não faça isso. ps5 ta uma completa ...,0,0
3,"hoje lançam o preço do ps5, tô só por saber qu...",1,0
4,"@1playergirl bom, nesse momento me sinto num h...",0,1
5,era tudo mentira! sony nega declaração da bloo...,0,1
6,microsoft vai se alavanca muito na proxima ger...,1,0
7,@kingkrysstan linde parece um ps5 uma geladeir...,0,0
8,não eu quero um ps5 https://t.co/2aqh0v93ou,0,1
9,"preciso de um ps5, me indiquem idosas para me ...",0,1


In [13]:
pd.crosstab(test['Relevância:'], test['Classificação'], normalize='index')

Classificação,0,1
Relevância:,,
0,0.569892,0.430108
1,0.514019,0.485981


___
### Concluindo

___
## Aperfeiçoamento:

Os trabalhos vão evoluir em conceito dependendo da quantidade de itens avançados:

* Limpar: \n, :, ", ', (, ), etc SEM remover emojis
* Corrigir separação de espaços entre palavras e emojis ou entre emojis e emojis
* Propor outras limpezas e transformações que não afetem a qualidade da informação ou classificação
* Criar categorias intermediárias de relevância baseadas na probabilidade: ex.: muito relevante, relevante, neutro, irrelevante, muito irrelevante (3 categorias: C, mais categorias conta para B)
* Explicar por que não posso usar o próprio classificador para gerar mais amostras de treinamento
* Propor diferentes cenários para Naïve Bayes fora do contexto do projeto
* Sugerir e explicar melhorias reais com indicações concretas de como implementar (indicar como fazer e indicar material de pesquisa)
* Montar um dashboard que realiza análise de sentimento e visualiza estes dados

___
## Referências

[Naive Bayes and Text Classification](https://arxiv.org/pdf/1410.5329.pdf)  **Mais completo**

[A practical explanation of a Naive Bayes Classifier](https://monkeylearn.com/blog/practical-explanation-naive-bayes-classifier/) **Mais simples**